In [1]:
class Variable:
    
    def __init__(self, data):
        self.data = data
        self.grad = None
        self.creator = None
        
    def set_creator(self, func):
        self.creator = func
    
    def backward(self):
        if self.grad is None:
            self.grad = np.ones_like(self.data)
        
        
        funcs = [self.creator]
        while funcs:
            f = funcs.pop()
            gys = [output.grad for output in f.outputs]
            gxs = f.backward(*gys)
            if not isinstance(gxs, tuple):
                gxs = (gxs, )
            
            for x, gx in zip(f.inputs, gxs):
                x.grad =gx
                if x.creator is not None:
                    funcs.append(x.creator)

In [3]:

class Function:
    def __call__(self, *inputs):
        xs = [x.data for x in inputs]
        ys = self.forward(*xs)
        if not isinstance(ys, tuple):
            ys = (ys,)
        outputs = [Variable(as_array(y)) for y in ys]
        
        for output in outputs:
            output.set_creator(self)
            
        self.inputs = inputs
        self.outputs = outputs
        
        return  outputs if len(outputs) > 1 else outputs[0]

In [4]:

class Add(Function):
    def forward(self, x0, x1):
        y = x0 + x1
        return y
    def backward(self, gy):
        return gy, gy

In [7]:
def as_array(x):
    if np.isscalar(x):
        return np.array((x))
    return x
def add(x0, x1):
    return Add()(x0, x1)

In [8]:
x = Variable(np.array(3.0))
y = add(x,x)
print('y', y.data)


y.backward()
print('x.grad', x.grad)

y 6.0
x.grad 1.0


In [9]:
#위의 결과를 보면  x = 3.0으로 설정한 후 더하였으니 6으로 더한값은 제대로 나왔다. 하지만 x.grad는 1이라는 잘못된 결과가 나왔습니다.
# 제대로 계산한다면  y = x+ x일때 y=2x 이니 미분값 2가 됩니다.

In [ ]:
class Variable:
    
    def __init__(self, data):
        self.data = data
        self.grad = None
        self.creator = None
        
    def set_creator(self, func):
        self.creator = func
    
    def backward(self):
        if self.grad is None:
            self.grad = np.ones_like(self.data)
        
        
        funcs = [self.creator]
        while funcs:
            f = funcs.pop()
            gys = [output.grad for output in f.outputs]
            gxs = f.backward(*gys)
            if not isinstance(gxs, tuple):
                gxs = (gxs, )
            
            for x, gx in zip(f.inputs, gxs):
                x.grad =gx   ##### 이부분이 문제 입니다.
                if x.creator is not None:
                    funcs.append(x.creator)

# 해결후


In [10]:
class Variable:
    
    def __init__(self, data):
        self.data = data
        self.grad = None
        self.creator = None
        
    def set_creator(self, func):
        self.creator = func
    
    def backward(self):
        if self.grad is None:
            self.grad = np.ones_like(self.data)
        
        
        funcs = [self.creator]
        while funcs:
            f = funcs.pop()
            gys = [output.grad for output in f.outputs]
            gxs = f.backward(*gys)
            if not isinstance(gxs, tuple):
                gxs = (gxs, )
            
            for x, gx in zip(f.inputs, gxs):
                #수정 부분 =========================== 
                if x.grad is None: 
                    x.grad = gx
                    
                else:
                    x.grad = x.grad + gx
                    
                # ====================================
                if x.creator is not None:
                    funcs.append(x.creator)

In [11]:
x = Variable(np.array(3.0))
y = add(x,x)
print('y', y.data)


y.backward()
print('x.grad', x.grad)

y 6.0
x.grad 2.0


In [12]:
x = Variable(np.array(3.0))
y = add(add(x,x), x)

print('y', y.data)


y.backward()
print('x.grad', x.grad)

y 9.0
x.grad 3.0
